<a href="https://colab.research.google.com/github/yashv04/Fantasy/blob/main/Fantasy_Team_CSK_vs_KKR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

In [ ]:
import pulp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [ ]:
df = pd.read_csv('ipl_2022_match1_csk_kkr.csv')
df.head(2)

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points
0,4061,45886,M Dhoni,NaN,CSK,NaN,WK,39.54,8.5,NaN,NaN,NaN,NaN
1,4061,45886,S Billings,NaN,KKR,NaN,WK,71.13,8.5,NaN,NaN,NaN,NaN


In [ ]:
df = df.join(pd.get_dummies(df[['Role', 'Team']]))

In [ ]:
df['Playing_xi'] = 1

In [ ]:
df.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_CSK,Team_KKR,Playing_xi
0,4061,45886,M Dhoni,NaN,CSK,NaN,WK,39.54,8.5,NaN,NaN,NaN,NaN,False,False,False,True,True,False,1
1,4061,45886,S Billings,NaN,KKR,NaN,WK,71.13,8.5,NaN,NaN,NaN,NaN,False,False,False,True,False,True,1
2,4061,45886,S Jackson,NaN,KKR,NaN,WK,5.80,8.0,NaN,NaN,NaN,NaN,False,False,False,True,False,True,1
3,4061,45886,N Jagadeesan,NaN,CSK,NaN,WK,5.24,8.0,NaN,NaN,NaN,NaN,False,False,False,True,True,False,1
4,4061,45886,S Iyer,NaN,KKR,NaN,BAT,86.62,9.5,NaN,NaN,NaN,NaN,False,True,False,False,False,True,1


In [ ]:
pickup_status = pulp.LpVariable.dicts('pickup_status', ((obj) for obj in df.index),cat='Binary')

#define objective function

mo = pulp.LpProblem('Sel maximization problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Sel'] for obj in df.index])

#define constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Credits'] for obj in df.index]) <= 100

#11 players
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Playing_xi'] for obj in df.index]) == 11

#team constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_CSK'] for obj in df.index]) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_CSK'] for obj in df.index]) <= 7

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_KKR'] for obj in df.index]) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_KKR'] for obj in df.index]) <= 7

# role constraints

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT'] for obj in df.index]) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT'] for obj in df.index]) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL'] for obj in df.index]) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL'] for obj in df.index]) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR'] for obj in df.index]) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR'] for obj in df.index]) <= 4

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK'] for obj in df.index]) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK'] for obj in df.index]) <= 4

In [ ]:
mo.solve()

1

In [ ]:
#output as a column in df
df['pickup_status'] = 0

for obj in df.index:
  if pickup_status[obj].varValue:
    df['pickup_status'][obj] = 1

In [ ]:
fdf = df[df['pickup_status']== 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False)
fdf

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_CSK,Team_KKR,Playing_xi,pickup_status
4,4061,45886,S Iyer,NaN,KKR,NaN,BAT,86.62,9.5,NaN,NaN,NaN,NaN,False,True,False,False,False,True,1,1
5,4061,45886,R Gaikwad,NaN,CSK,NaN,BAT,83.45,9.5,NaN,NaN,NaN,NaN,False,True,False,False,True,False,1,1
6,4061,45886,A Rayudu,NaN,CSK,NaN,BAT,45.80,9.0,NaN,NaN,NaN,NaN,False,True,False,False,True,False,1,1
7,4061,45886,N Rana,NaN,KKR,NaN,BAT,48.61,9.0,NaN,NaN,NaN,NaN,False,True,False,False,False,True,1,1
1,4061,45886,S Billings,NaN,KKR,NaN,WK,71.13,8.5,NaN,NaN,NaN,NaN,False,False,False,True,False,True,1,1
18,4061,45886,A Russell,NaN,KKR,NaN,AR,61.68,10.0,NaN,NaN,NaN,NaN,True,False,False,False,False,True,1,1
20,4061,45886,R Jadeja,NaN,CSK,NaN,AR,80.28,9.5,NaN,NaN,NaN,NaN,True,False,False,False,True,False,1,1
22,4061,45886,V Iyer,NaN,KKR,NaN,AR,52.30,9.0,NaN,NaN,NaN,NaN,True,False,False,False,False,True,1,1
30,4061,45886,P Cummins,NaN,KKR,NaN,BOWL,46.15,9.0,NaN,NaN,NaN,NaN,False,False,True,False,False,True,1,1
31,4061,45886,D Bravo,NaN,CSK,NaN,BOWL,68.38,8.5,NaN,NaN,NaN,NaN,False,False,True,False,True,False,1,1


In [ ]:
sum(fdf.Credits)

100.0

In [ ]:
fdf.to_csv('optimal_team.csv', index=False)

In [ ]:
from google.colab import files
files.download('optimal_team.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>